In [ ]:
%matplotlib widget

In [ ]:
from glob import glob
import numpy as np
import pandas as pd
import flammkuchen as fl
from split_dataset import SplitDataset
from bouter import Experiment
from fimpy.pipeline.general import calc_f0, dff
from motions.utilities import stim_vel_dir_dataframe, quantize_directions
from scipy.interpolate import interp1d 
from scipy.signal import convolve2d
import colorspacious
import matplotlib.pyplot as plt
import json
from pathlib import Path

In [ ]:
master = Path(r"Z:\Hagar\E0040\v31\post ablation 1d")
fish_list = list(master.glob("*_f*"))
path = fish_list[4]
print(path)

#path = Path(r"Z:\Hagar\crazy fish\230302_f0\e0040_v10")

In [ ]:
exp = glob(str(path / "*behavior*"))[0]
suite2p_brain = fl.load(path / "data_from_suite2p_cells_brain.h5")
in_brain_idx = suite2p_brain['coords_idx']
traces = fl.load(path / "filtered_traces.h5", "/undetr")[:, in_brain_idx]


In [ ]:
np.shape(traces)

In [ ]:
coords = fl.load(path / 'registration' / "mov_coords_transformed.h5")[in_brain_idx]

In [ ]:
with open(next(Path(path).glob("*metadata.json")), "r") as f:
        metadata = json.load(f)
lsconfig = metadata["imaging"]["microscope_config"]['lightsheet']['scanning']
z_tot_span = lsconfig["z"]["piezo_max"] - lsconfig["z"]["piezo_min"]
n_planes = lsconfig["triggering"]["n_planes"]
z_res = z_tot_span / n_planes
res = [1, 1, 1]

In [ ]:
sensory_regressors = fl.load(path / "sensory_regressors.h5", "/regressors")
titles = ['right', 'backward right', 'backward', 'backward left', 'left', 'forward left', 'forward', 'forward right']

In [ ]:
fig, axs = plt.subplots(2, 4, figsize=(15, 5), gridspec_kw={'height_ratios': [1, 3]}, sharex=True)

In [ ]:
for direction in range(0, 8, 2):
    print(direction)
    #traces = traces[:1600]
    current_dir = np.asarray(sensory_regressors.iloc[:, direction])
    num_traces = np.shape(traces)[1]
        
    reg_corr = np.zeros((num_traces))
    for i in range(num_traces):
        reg_corr[i] = np.corrcoef(current_dir, traces[:, i])[0,1]
       
        
    mp_ind = np.argsort(np.abs(reg_corr))
    
    # sagittal
    axs[1,direction//2].scatter(coords[mp_ind,2]*res[2] + 550, coords[mp_ind,1]*res[1], c=reg_corr[mp_ind], s=2, alpha=0.8, cmap='coolwarm', vmin=-1, vmax=1)
    # horizontal
    axs[1,direction//2].scatter(coords[mp_ind,0]*res[0], coords[mp_ind,1]*res[1], c=reg_corr[mp_ind], s=2, alpha=0.8, cmap='coolwarm', vmin=-1, vmax=1)
    # frontal
    axs[0,direction//2].scatter(coords[mp_ind,0]*res[0], coords[mp_ind,2]*res[2], c=reg_corr[mp_ind], s=2, alpha=0.8, cmap='coolwarm', vmin=-1, vmax=1)

    axs[0,direction//2].spines['right'].set_visible(False)
    axs[0,direction//2].spines['top'].set_visible(False)

    axs[1,direction//2].spines['right'].set_visible(False)
    axs[1,direction//2].spines['top'].set_visible(False)

    axs[0,direction//2].set_title(titles[direction])
    
    axs[1,direction//2].invert_yaxis()
    
fig.subplots_adjust(left=0.05, wspace=0.1)

In [ ]:
fig.subplots_adjust(left=0.05, wspace=0.1)

In [ ]:
file_name = "corr map all directions morphed.jpg"
fig.savefig(path / file_name, dpi=300)

In [ ]:
np.shape(current_dir)

In [ ]:
reg_values = current_dir @ traces[:, :] 

In [ ]:
num_traces = np.shape(traces)[1]
print(num_traces)
reg_corr = np.zeros((num_traces))
for i in range(num_traces):
    reg_corr[i] = np.corrcoef(current_dir, traces[:, i])[0,1]

In [ ]:
coords_ib = coords[in_brain_idx]
reg_values_ib = reg_values[in_brain_idx]
reg_corr_ib = reg_corr[in_brain_idx]

In [ ]:
titles = ['right', 'backward right', 'backward', 'backward left', 'left', 'forward left', 'forward', 'forward right', ]

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(6, 6), gridspec_kw={'width_ratios': [6, 2], 'height_ratios': [1, 2]})
mp_ind = np.argsort(np.abs(reg_corr_ib))
axs[1,0].scatter(coords_ib[mp_ind,2]*0.6, coords_ib[mp_ind,1]*.6, c=reg_corr_ib[mp_ind], s=2, alpha=0.8, cmap='coolwarm', vmin=-1, vmax=1)
axs[1,1].scatter(coords_ib[mp_ind,0]*z_res, coords_ib[mp_ind,1]*0.6, c=reg_corr_ib[mp_ind], alpha=0.8, cmap='coolwarm', vmin=-1, vmax=1)
axs[0,0].scatter(coords_ib[mp_ind,2]*0.6, coords_ib[mp_ind,0]*z_res, c=reg_corr_ib[mp_ind], alpha=0.8, cmap='coolwarm', vmin=-1, vmax=1)

axs[0,0].spines['right'].set_visible(False)
axs[0,0].spines['top'].set_visible(False)

axs[1,1].spines['right'].set_visible(False)
axs[1,1].spines['top'].set_visible(False)

axs[1,0].spines['right'].set_visible(False)
axs[1,0].spines['top'].set_visible(False)

axs[0,1].axis('off')

fig.suptitle(titles[direction])

In [ ]:
file_name = "corr map " + titles[direction] + ".jpg"
fig.savefig(path / file_name, dpi=300)